# Goal
Programmatically run PAMl analysis with different input alignment, tree and options, and collect the results for output. This helps with reproducibility and makes documenting the analysis easier.

# Approach
Use `Biopython`'s integration for PAML.

In [39]:
from Bio.Phylo.PAML import codeml
from scipy.stats import chi2
import os
import csv

In [4]:
# store the script directory
script_dir = os.path.abspath('')
print(script_dir)

/Users/bhe2/Documents/work/current/C037-Cand-auris-adhesin/02-case-studies/09-natural-selection/script


# p1-414
## Site test

### M0,1,2,7,8 (codonfreq=1)

In [10]:
# to allow for this code chunk to be run out of order, first switch back to the script dir
os.chdir(script_dir)
# change to the first segment analysis dir
working_dir = "../output/paml/B8441-OG-part/p1-414/site-cf1/"
os.chdir(working_dir)

In [4]:
cml = codeml.Codeml()
cml.working_dir = "./"
cml.alignment = "../B8441-OG-Hil-PF11765-1-414.nuc"
cml.tree = "../p1-414.nwk"
cml.out_file = "mlc"

In [7]:
cml.set_options(
    noisy = 2,
    verbose = 0,
    seqtype = 1,
    CodonFreq = 1,
    clock = 0,
    model = 0, # one ratio for all branches
    NSsites = [0,1,2,7,8],
    icode = 8,
    fix_kappa = 0,
    kappa = 2,
    fix_omega = 0,
    omega = .4,
    cleandata = 1,
    fix_blength = 0
)

In [8]:
cml.ctl_file = "codeml.ctl"
cml.write_ctl_file()

In [9]:
res = cml.run()

### M8a (codonfreq=1)

In [11]:
# to allow for this code chunk to be run out of order, first switch back to the script dir
os.chdir(script_dir)
# change to the first segment analysis dir
working_dir = "../output/paml/B8441-OG-part/p1-414/site-M8a-cf1/"
os.chdir(working_dir)

In [12]:
cml = codeml.Codeml()
cml.working_dir = "./"
cml.alignment = "../B8441-OG-Hil-PF11765-1-414.nuc"
cml.tree = "../p1-414.nwk"
cml.out_file = "mlc"

In [13]:
cml.set_options(
    noisy = 2,
    verbose = 0,
    seqtype = 1,
    CodonFreq = 1,
    clock = 0,
    model = 0, # one ratio for all branches
    NSsites = [8],
    icode = 8,
    fix_kappa = 0,
    kappa = 2,
    fix_omega = 1,
    omega = 1,
    cleandata = 1,
    fix_blength = 0
)

In [14]:
cml.ctl_file = "codeml.ctl"
cml.write_ctl_file()

In [15]:
res = cml.run()

### M0,1,2,7,8 (codonfreq=2)

In [16]:
# to allow for this code chunk to be run out of order, first switch back to the script dir
os.chdir(script_dir)
# change to the first segment analysis dir
working_dir = "../output/paml/B8441-OG-part/p1-414/site-cf2/"
os.chdir(working_dir)

In [17]:
cml = codeml.Codeml()
cml.working_dir = "./"
cml.alignment = "../B8441-OG-Hil-PF11765-1-414.nuc"
cml.tree = "../p1-414.nwk"
cml.out_file = "mlc"

In [18]:
cml.set_options(
    noisy = 2,
    verbose = 0,
    seqtype = 1,
    CodonFreq = 2,
    clock = 0,
    model = 0, # one ratio for all branches
    NSsites = [0,1,2,7,8],
    icode = 8,
    fix_kappa = 0,
    kappa = 2,
    fix_omega = 0,
    omega = .4,
    cleandata = 1,
    fix_blength = 0
)

In [19]:
cml.ctl_file = "codeml.ctl"
cml.write_ctl_file()

In [20]:
res = cml.run()

### M8a (codonfreq=2)

In [21]:
# to allow for this code chunk to be run out of order, first switch back to the script dir
os.chdir(script_dir)
# change to the first segment analysis dir
working_dir = "../output/paml/B8441-OG-part/p1-414/site-M8a-cf2/"
os.chdir(working_dir)

In [22]:
cml = codeml.Codeml()
cml.working_dir = "./"
cml.alignment = "../B8441-OG-Hil-PF11765-1-414.nuc"
cml.tree = "../p1-414.nwk"
cml.out_file = "mlc"

In [23]:
cml.set_options(
    noisy = 2,
    verbose = 0,
    seqtype = 1,
    CodonFreq = 2,
    clock = 0,
    model = 0, # one ratio for all branches
    NSsites = [8],
    icode = 8,
    fix_kappa = 0,
    kappa = 2,
    fix_omega = 1,
    omega = 1,
    cleandata = 1,
    fix_blength = 0
)

In [24]:
cml.ctl_file = "codeml.ctl"
cml.write_ctl_file()

In [25]:
res = cml.run()

### Site test result

In [114]:
# read site model results into a dictionary
site_res = {
    'site-cf1': codeml.read('site-cf1/mlc'),
    'site-cf2': codeml.read('site-cf2/mlc'),
    'site-M8a-cf1': codeml.read('site-M8a-cf1/mlc'),
    'site-M8a-cf2': codeml.read('site-M8a-cf2/mlc')
}

In [130]:
# function to extract the useful information from each model fit
def get_site_model_fit(res):
    out = [] # list of lists to store the model output
    codon = res['codon model'] # codon model used
    for m, fit in res['NSsites'].items():
        # model number
        M = f'M{m}'
        # get the model description
        desc = dict.get(fit, 'description')
        # get the number of parameters
        par = dict.get(fit, 'parameters')
        l = len(par['parameter list'].split(' '))
        # get the log likelihood values
        lnL = dict.get(fit, 'lnL')
        # get the kappa estimate
        k = dict.get(par, 'kappa')
        # get the model-specific set of parameters
        omega = ''
        # separately deal with the different site models
        if m == 0:
            omega = f'w={par["omega"]:.3f}'
        elif m < 7:
            site_class = dict.get(par, 'site classes')
            #print(site_class)
            for i, v in site_class.items():
                omega += f'p{i}={v["proportion"]:.3f}, w{i}={float(v["omega"]):.3f}; '
        elif m == 7:
            omega = f'p={par["p"]:.3f}, q={par["q"]:.3f}'
        elif m == 8:
            omega = f'p0={par["p0"]:.3f}, p={par["p"]:.3f}, q={par["q"]:.3f}; p1={1-par["p0"]:.3f}, w={par["w"]:.3f}'
            if par['w'] == 1.0: # model M8a
                M = 'M8a'
                
        # print the results to the screen
        # print(f'{M}: {desc} has {l} parameters, lnL = {lnL:.3f}, kappa = {k:.3f}, \n\tsite classes: {omega}')
        # append the results as a row
        out.append([codon, M, l, lnL, k, omega, desc])
    return(out)


In [131]:
# process the data and write the output to a csv file
data_rows = []
for t in site_res.values():
    data_rows += get_site_model_fit(t)
fields = ['codonfreq', 'model', 'npar', 'lnL', 'kappa', 'omega', 'description']
with open("20220727-p1-414-site-model-summary-table.tsv", "w") as f:
    write = csv.writer(f, delimiter = "\t")
    write.writerow(fields)
    write.writerows(data_rows)

No evidence was found to support positive selection by any of the tests.

## Branch test

### one ratio (codonfreq = 1)

In [ ]:
# to allow for this code chunk to be run out of order, first switch back to the script dir
os.chdir(script_dir)
# change to the first segment analysis dir
working_dir = "../output/paml/B8441-OG-part/p1-414/branch-1r-cf1/"
os.chdir(working_dir)

In [40]:
cml = codeml.Codeml()
cml.working_dir = "./"
cml.alignment = "../B8441-OG-Hil-PF11765-1-414.nuc"
cml.tree = "../p1-414.nwk"
cml.out_file = "mlc"

In [41]:
cml.set_options(
    noisy = 2,
    verbose = 0,
    seqtype = 1,
    CodonFreq = 1,
    model = 0,
    NSsites = [0],
    icode = 8,
    fix_kappa = 0,
    kappa = 2,
    fix_omega = 0,
    omega = .4,
    cleandata = 1,
    fix_blength = 0
)

In [54]:
cml.ctl_file = "codeml.ctl"
cml.write_ctl_file()

In [55]:
res = cml.run()

### free ratio (codonfreq = 1)

In [ ]:
# to allow for this code chunk to be run out of order, first switch back to the script dir
os.chdir(script_dir)
# change to the first segment analysis dir
working_dir = "../output/paml/B8441-OG-part/p1-414/branch-freer-cf1/"
os.chdir(working_dir)

In [69]:
cml = codeml.Codeml()
cml.working_dir = "./"
cml.alignment = "../B8441-OG-Hil-PF11765-1-414.nuc"
cml.tree = "../p1-414.nwk"
cml.out_file = "mlc"

In [74]:
cml.set_options(
    noisy = 2,
    verbose = 0,
    seqtype = 1,
    CodonFreq = 1,
    clock = 0,
    model = 1,
    NSsites = [0],
    icode = 8,
    fix_kappa = 0,
    kappa = 2,
    fix_omega = 0,
    omega = .4,
    cleandata = 1,
    fix_blength = 0
)

In [75]:
cml.ctl_file = "codeml.ctl"
cml.write_ctl_file()

In [76]:
res = cml.run()

### one ratio (codonfreq = 2)

In [ ]:
# to allow for this code chunk to be run out of order, first switch back to the script dir
os.chdir(script_dir)
# change to the first segment analysis dir
working_dir = "../output/paml/B8441-OG-part/p1-414/branch-1r-cf2/"
os.chdir(working_dir)

In [40]:
cml = codeml.Codeml()
cml.working_dir = "./"
cml.alignment = "../B8441-OG-Hil-PF11765-1-414.nuc"
cml.tree = "../p1-414.nwk"
cml.out_file = "mlc"

In [84]:
cml.set_options(
    noisy = 2,
    verbose = 0,
    seqtype = 1,
    CodonFreq = 2,
    model = 0,
    NSsites = [0],
    icode = 8,
    fix_kappa = 0,
    kappa = 2,
    fix_omega = 0,
    omega = .4,
    cleandata = 1,
    fix_blength = 0
)

In [85]:
cml.ctl_file = "codeml.ctl"
cml.write_ctl_file()

In [86]:
res = cml.run()

### free ratio (codonfreq = 2)

In [ ]:
# to allow for this code chunk to be run out of order, first switch back to the script dir
os.chdir(script_dir)
# change to the first segment analysis dir
working_dir = "../output/paml/B8441-OG-part/p1-414/branch-freer-cf2/"
os.chdir(working_dir)

In [89]:
cml = codeml.Codeml()
cml.working_dir = "./"
cml.alignment = "../B8441-OG-Hil-PF11765-1-414.nuc"
cml.tree = "../p1-414.nwk"
cml.out_file = "mlc"

In [90]:
cml.set_options(
    noisy = 2,
    verbose = 0,
    seqtype = 1,
    CodonFreq = 2,
    clock = 0,
    model = 1,
    NSsites = [0],
    icode = 8,
    fix_kappa = 0,
    kappa = 2,
    fix_omega = 0,
    omega = .4,
    cleandata = 1,
    fix_blength = 0
)

In [91]:
cml.ctl_file = "codeml.ctl"
cml.write_ctl_file()

In [92]:
res = cml.run()

### Free vs one-omega models
First, we reimport all the model results

In [40]:
# to allow for this code chunk to be run out of order, first switch back to the script dir
os.chdir(script_dir)
# change to the first segment analysis dir
working_dir = "../output/paml/B8441-OG-part/p1-414/"
os.chdir(working_dir)
branch_res = {
    'branch-1r-cf1': codeml.read('branch-1r-cf1/mlc'),
    'branch-1r-cf2': codeml.read('branch-1r-cf2/mlc'),
    'branch-freer-cf1': codeml.read('branch-freer-cf1/mlc'),
    'branch-freer-cf2': codeml.read('branch-freer-cf2/mlc')
}

In [6]:
def get_ll(res):
    return(res['NSsites'][0]['lnL'])

def get_np(res):
    par = res['NSsites'][0]['parameters']['parameter list']
    return(len(par.split(' ')))

In [45]:
dl_cf1 = get_ll(branch_res['branch-freer-cf1']) - get_ll(branch_res['branch-1r-cf1'])
df_cf1 = get_np(branch_res['branch-freer-cf1']) - get_np(branch_res['branch-1r-cf1'])
p_cf1 = 1 - chi2.cdf(2*dl_cf1, df = df_cf1)
print("Testing one ratio vs free ratio models, with CodonFreq=1")
print(f'df={df_cf1}, 2*∆lnL={2*dl_cf1:.2f}, p={p_cf1:.2e}')

Testing one ratio vs free ratio models, with CodonFreq=1
df=14, 2*∆lnL=55.73, p=6.47e-07


In [46]:
dl_cf2 = get_ll(branch_res['branch-freer-cf2']) - get_ll(branch_res['branch-1r-cf2'])
df_cf2 = get_np(branch_res['branch-freer-cf2']) - get_np(branch_res['branch-1r-cf2'])
p_cf2 = 1 - chi2.cdf(2*dl_cf2, df = df_cf2)
# p_cf2 = 1.427e-02 # calculated with the chi2 program from PAML package
print("Testing one ratio vs free ratio models, with CodonFreq=1")
print(f'df={df_cf2}, 2*∆lnL={2*dl_cf2:.2f}, p={p_cf2:.2e}')

Testing one ratio vs free ratio models, with CodonFreq=1
df=14, 2*∆lnL=27.99, p=1.43e-02


there is thus strong evidence for non-equal dN/dS along the branches. But this test is almost always significant because the null hypothesis of homogeneous dN/dS across the entire tree is not realistic under most cases.

I plotted the free ratio model estimates on the gene tree using the `ggtree` package in R:
![free ratio estimates](../output/figure/20220727-B8441-OG-p1-414-branch-freer-tree.png)

By comparing the results obtained with CodonFreq=2 or CodonFreq=1, I see that two of the three branches identified as having elevated dN/dS by the F3x4 model was also implicated by the F1x4 model. Given Ziheng's [suggestion](https://groups.google.com/g/pamlsoftware/c/i7-NFSgnhq8/m/80rWE37kBgAJ), I decided to focus on the F3x4 model result and test the three branches together and separately (labeled as ω1, ω2 and ω3, respectively).

My next goal is to separately test whether there are statistical support for these two branches having a dN/dS > 1.

### F1x4, two ratio (w0, w1=w2=w3=w4)

In [73]:
# to allow for this code chunk to be run out of order, first switch back to the script dir
os.chdir(script_dir)
# change to the first segment analysis dir
working_dir = "../output/paml/B8441-OG-part/p1-414/branch-2r_all-cf1/"
if not os.path.exists(working_dir):
    os.mkdir(working_dir)
os.chdir(working_dir)

In [48]:
cml = codeml.Codeml()
cml.working_dir = "./"
cml.alignment = "../B8441-OG-Hil-PF11765-1-414.nuc"
cml.tree = "../p1-414-2r_all-cf1.nwk"
cml.out_file = "mlc"

In [49]:
cml.set_options(
    noisy = 2,
    verbose = 0,
    seqtype = 1,
    CodonFreq = 1, # F1x4 codon
    model = 2,     # 2 or more dN/dS ratios for branches
    NSsites = [0], # one site class 
    icode = 8,     # yeast alternative nuclear code
    fix_kappa = 0,
    kappa = 2,
    fix_omega = 0,
    omega = .4,
    cleandata = 1,
    fix_blength = 0
)

In [50]:
cml.ctl_file = "codeml.ctl"
cml.write_ctl_file()

In [51]:
res = cml.run()

In [66]:
# define a few helper functions
def print_branch_test(res1, res0):
    """
    extract and print the most important branch test result parameters, including
    codonmodel, lnL, npar, omega_tree
    also perform a log likelihood ratio test comparing the alternative with the null model
    designated as res1 and res0
    """
    # get the lnL and npar for both models
    lnL1 = get_ll(res1); npar1 = get_np(res1)
    lnL0 = get_ll(res0); npar0 = get_np(res0)
    dlnL = lnL1 - lnL0; dnpar = npar1 - npar0
    llr_P = 1 - chi2.cdf(x = 2*dlnL, df = dnpar)
    # get the parameter lists for both models
    res1_fit = res1["NSsites"][0]; res1_par = res1_fit["parameters"]
    res0_fit = res0["NSsites"][0]; res0_par = res0_fit["parameters"]
    # print useful model output
    print(f'Main result (codon model = {res1["codon model"]})\n\
    lnL: {lnL1}; np: {npar1}; kappa: {res1_par["kappa"]:.3f}; w: {res1_par["omega"]}\n\
    omega tree: {res1_fit["omega tree"]}')
    print(f'Null model (codon model = {res0["codon model"]})\n\
    lnL: {lnL0}; np: {npar0}; kappa: {res0_par["kappa"]:.3f}; w: {res0_par["omega"]}\n\
    2*dlnL = {2*dlnL:.3f}, df = {dnpar}, LLR P-value = {llr_P:.2e}')

In [74]:
branch_res['branch-2r_all-cf1'] = codeml.read("mlc")
print_branch_test(branch_res['branch-2r_all-cf1'], branch_res['branch-1r-cf1'])

Main result (codon model = F1x4)
    lnL: -3421.85936; np: 18; kappa: 1.583; w: [0.12695, 1.77137]
    omega tree: ((Hil7 #0.126954 , (((Hil1 #0.126954 , Hil2 #0.126954 ) #0.126954 , (Hil4 #0.126954 , Hil3 #0.126954 ) #0.126954 ) #1.77137 , (Hil8 #0.126954 , Hil6 #0.126954 ) #1.77137 ) #1.77137 ) #0.126954 , Hil5 #1.77137 , OG #0.126954 );
Null model (codon model = F1x4)
    lnL: -3427.952669; np: 17; kappa: 1.592; w: 0.13685
    2*dlnL = 12.187, df = 1, LLR P-value = 4.81e-04


### F1x4, two ratio constrained (w0, w1=w2=w3=w4=1)

In [76]:
# to allow for this code chunk to be run out of order, first switch back to the script dir
os.chdir(script_dir)
# change to the first segment analysis dir
working_dir = "../output/paml/B8441-OG-part/p1-414/branch-2r_alleq1-cf1/"
if not os.path.exists(working_dir):
    os.mkdir(working_dir)
os.chdir(working_dir)

In [69]:
cml = codeml.Codeml()
cml.working_dir = "./"
cml.alignment = "../B8441-OG-Hil-PF11765-1-414.nuc"
cml.tree = "../p1-414-2r_all-cf1.nwk"
cml.out_file = "mlc"

In [70]:
cml.set_options(
    noisy = 2,
    verbose = 0,
    seqtype = 1,
    CodonFreq = 1, # F1x4 codon
    model = 2,     # 2 or more dN/dS ratios for branches
    NSsites = [0], # one site class 
    icode = 8,     # yeast alternative nuclear code
    fix_kappa = 0,
    kappa = 2,
    fix_omega = 1,
    omega = 1,
    cleandata = 1,
    fix_blength = 0
)

In [71]:
cml.ctl_file = "codeml.ctl"
cml.write_ctl_file()

In [72]:
res = cml.run()

In [77]:
branch_res['branch-2r_alleq1-cf1'] = codeml.read("mlc")
print_branch_test(branch_res['branch-2r_all-cf1'], branch_res['branch-2r_alleq1-cf1'])

Main result (codon model = F1x4)
    lnL: -3421.85936; np: 18; kappa: 1.583; w: [0.12695, 1.77137]
    omega tree: ((Hil7 #0.126954 , (((Hil1 #0.126954 , Hil2 #0.126954 ) #0.126954 , (Hil4 #0.126954 , Hil3 #0.126954 ) #0.126954 ) #1.77137 , (Hil8 #0.126954 , Hil6 #0.126954 ) #1.77137 ) #1.77137 ) #0.126954 , Hil5 #1.77137 , OG #0.126954 );
Null model (codon model = F1x4)
    lnL: -3421.992047; np: 17; kappa: 1.581; w: [0.1292, 1.0]
    2*dlnL = 0.265, df = 1, LLR P-value = 6.06e-01


### two ratio (w0, w1=w2=w3, F3x4)

In [79]:
# to allow for this code chunk to be run out of order, first switch back to the script dir
os.chdir(script_dir)
# change to the first segment analysis dir
working_dir = "../output/paml/B8441-OG-part/p1-414/branch-2r_all-cf2/"
if not os.path.exists(working_dir):
    os.mkdir(working_dir)
os.chdir(working_dir)

In [80]:
cml = codeml.Codeml()
cml.working_dir = "./"
cml.alignment = "../B8441-OG-Hil-PF11765-1-414.nuc"
cml.tree = "../p1-414-2r_all-cf2.nwk"
cml.out_file = "mlc"

In [81]:
cml.set_options(
    noisy = 2,
    verbose = 0,
    seqtype = 1,
    CodonFreq = 2, # F3x4 model
    model = 2,     # 2 or more dN/dS ratios for branches
    NSsites = [0], # one site class 
    icode = 8,     # yeast alternative nuclear code
    fix_kappa = 0,
    kappa = 2,
    fix_omega = 0,
    omega = .4,
    cleandata = 1,
    fix_blength = 0
)

In [82]:
cml.ctl_file = "codeml.ctl"
cml.write_ctl_file()

In [83]:
res = cml.run()

In [84]:
branch_res['branch-2r_all-cf2'] = codeml.read("mlc")
print_branch_test(branch_res['branch-2r_all-cf2'], branch_res['branch-1r-cf2'])

Main result (codon model = F3x4)
    lnL: -3360.672152; np: 18; kappa: 1.316; w: [0.0292, 0.00277]
    omega tree: ((Hil7 #0.0291987 , (((Hil1 #0.0291987 , Hil2 #0.0291987 ) #0.0291987 , (Hil4 #0.0291987 , Hil3 #0.0291987 ) #0.0291987 ) #0.0291987 , (Hil8 #0.0291987 , Hil6 #0.0291987 ) #0.00277474 ) #0.00277474 ) #0.00277474 , Hil5 #0.0291987 , OG #0.0291987 );
Null model (codon model = F3x4)
    lnL: -3362.613054; np: 17; kappa: 1.342; w: 0.01825
    2*dlnL = 3.882, df = 1, LLR P-value = 4.88e-02


Note that the foreground lineages in the alternative model, which were selected based on them having significantly elevated dN/dS in the free ratio model, actually had a lower dN/dS estimate than the background. This is difficult to interpret.